# Errores y excepciones

Hasta ahora los mensajes de error no habían sido más que mencionados, pero al probar los ejemplos vimos algunos. Hay (al menos) dos tipos diferentes de errores: errores de sintaxis y excepciones

## Errores de sintaxis

Los errores de sintaxis, también conocidos como errores de interpretación, son quizás el tipo de queja más común cuando se esta aprendiendo Python

In [1]:
while True print('Hola mundo')

SyntaxError: invalid syntax (<ipython-input-1-52661a6c97e8>, line 1)

El intérprete repite la línea culpable y muestra una pequeña **‘flecha’** que apunta al primer lugar donde se detectó el error. Se muestran tambien el nombre del archivo y el número de línea para identificar mejor el lugar del error en caso de que la entrada venga de un programa

## Excepciones

Incluso si la declaración o expresión es sintácticamente correcta, puede generar un error cuando se intenta ejecutarla. Los errores detectados durante la ejecución se llaman **excepciones**, y no son incondicionalmente fatales

In [2]:
10 * (1/0)

ZeroDivisionError: division by zero

In [3]:
4 + spam*3

NameError: name 'spam' is not defined

In [4]:
'2' + 2

TypeError: Can't convert 'int' object to str implicitly

La última línea de los mensajes de error indica qué sucedió. Las excepciones vienen de distintos tipos, y el tipo se imprime como parte del mensaje. La cadena mostrada como tipo de la excepción es el nombre de la excepción predefinida que ocurrió. Esto es verdad para todas las excepciones predefinidas del intérprete, pero no necesita ser verdad para excepciones definidas por el usuario.

El resto de la línea provee un detalle basado en el tipo de la excepción y qué la causó.

La parte anterior del mensaje de error muestra el contexto donde la excepción sucedió, en la forma de un trazado del error listando líneas fuente; sin embargo, no mostrará líneas leídas desde la entrada estándar.

## Manejando excepciones

Es posible escribir programas que manejen determinadas excepciones. El siguiente ejemplo pide al usuario una entrada hasta que ingrese un entero válido, pero permite al usuario interrumpir el programa (usando Control-C o lo que sea que el sistema operativo soporte). 

Una interrupción generada por el usuario se señaliza generando la excepción **KeyboardInterrupt**

In [8]:
while True:
    try:
        x = int(input("Por favor ingrese un número: "))
        break
    except ValueError:
        print("Oops! No era válido. Intente nuevamente...")

Por favor ingrese un número: 3


La declaración **try** funciona de la siguiente manera:

* Primero, se ejecuta el bloque **try** (el código entre las declaración **try** y **except**).
* Si no ocurre ninguna excepción, el bloque **except** se saltea y termina la ejecución de la declaración **try**.
* Si ocurre una excepción durante la ejecución del bloque **try**, el resto del bloque se saltea. Luego, si su tipo coincide con la excepción nombrada luego de la palabra reservada **except**, se ejecuta el bloque **except**, y la ejecución continúa luego de la declaración **try**.
* Si ocurre una excepción que no coincide con la excepción nombrada en el **except**, esta se pasa a declaraciones **try** de más afuera; si no se encuentra nada que la maneje, es una excepción no manejada, y la ejecución se frena con un mensaje como los mostrados arriba.

Una declaración **try** puede tener más de un **except**, para especificar manejadores para distintas excepciones. A lo sumo un manejador será ejecutado. Sólo se manejan excepciones que ocurren en el correspondiente **try**, no en otros manejadores del mismo **try**. Un **except** puede nombrar múltiples excepciones usando paréntesis

In [9]:
except (RuntimeError, TypeError, NameError):
    pass

SyntaxError: invalid syntax (<ipython-input-9-2ba0955f2819>, line 1)

El último **except** puede omitir nombrar qué excepción captura, para servir como comodín. Esto se debe usar con extremo cuidado, ya que de esta manera es fácil ocultar un error real de programación. También puede usarse para mostrar un mensaje de error y luego re-generar la excepción

In [3]:
import sys

try:
    f = open('../datos/excepciones/miarchivo.txt')
    s = f.readline()
    i = int(s.strip())
    print(i)
except OSError as err:
    print("Error OS: {0}".format(err))
except ValueError:
    print("No pude convertir el dato a un entero.")
except:
    print("Error inesperado:", sys.exc_info()[0])
    raise

1


Las declaraciones **try ... except** tienen un bloque **else** opcional, el cual, cuando está presente, debe seguir a los except. Es útil para aquel código que debe ejecutarse si el bloque **try** no genera una excepción

In [4]:
for arg in sys.argv[2:]:
    try:
        f = open(arg, 'r')
    except IOError:
        print('no pude abrir', arg)
    else:
        print(arg, 'tiene', len(f.readlines()), 'lineas')
        f.close()

/run/user/1000/jupyter/kernel-e912da1d-1894-4145-881c-d3ad3bc3cf42.json tiene 12 lineas


El uso de **else** es mejor que agregar código adicional en el **try** porque evita capturar accidentalmente una excepción que no fue generada por el código que está protegido por la declaración **try ... except**.

Cuando ocurre una excepción, puede tener un valor asociado, también conocido como el argumento de la excepción. La presencia y el tipo de argumento depende del tipo de excepción.

El **except** puede especificar una variable luego del nombre de excepción. La variable se vincula a una instancia de excepción con los argumentos almacenados en **instance.args**. Por conveniencia, la instancia de excepción define **__str__()** para que se pueda mostrar los argumentos directamente, sin necesidad de hacer referencia a **.args**

In [16]:
try:
    raise Exception('spam', 'eggs')
except Exception as inst:
    print(type(inst))    # la instancia de excepción
    print(inst.args)     # argumentos guardados en .args
    print(inst)          # __str__ permite imprimir args directamente,
                         # pero puede ser cambiado en subclases de la exc
    x, y = inst.args     # desempacar argumentos
    print('x =', x)
    print('y =', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x = spam
y = eggs


Si una excepción tiene argumentos, estos se imprimen como la última parte del mensaje para las excepciones que no están manejadas.

Los manejadores de excepciones no maejan solamente las excepciones que ocurren en el bloque **try**, también manejan las excepciones que ocurren dentro de las funciones que se llaman dentro del bloque **try**

In [17]:
def esto_falla():
    x = 1/0
try:
    esto_falla()
except ZeroDivisionError as err:
    print('Manejando error en tiempo de ejecución:', err)

Manejando error en tiempo de ejecución: division by zero


## Levantando excepciones

La declaración **raise** permite al programador forzar a que ocurra una excepción específica

In [1]:
raise NameError('Hola')

NameError: Hola

El único argumento a **raise** indica la excepción a generarse. Tiene que ser o una instancia de excepción, o una clase de excepción (una clase que hereda de Exception).

Si se necesita determinar cuando una excepción fue lanzada pero no se quiere manejar, una forma simplificada de la instrucción **raise** te permite relanzarla

In [2]:
try:
    raise NameError('Hola')
except NameError:
    print('Voló una excepción!')
    raise

Voló una excepción!


NameError: Hola

## Definiendo acciones de limpieza

La declaración **try** tiene otra cláusula opcional que intenta definir acciones de limpieza que deben ser ejecutadas bajo ciertas circunstancias.

In [8]:
try:
    raise KeyboardInterrupt
finally:
    print('Chau, mundo!')

Chau, mundo!


KeyboardInterrupt: 

Una cláusula finally siempre es ejecutada antes de salir de la declaración **try**, ya sea que una excepción haya ocurrido o no. Cuando ocurre una excepción en la cláusula **try** y no fue manejada por una cláusula **except** (o ocurrió en una cláusula except o else), es relanzada luego de que se ejecuta la cláusula **finally**. El finally es también ejecutado a la salida cuando cualquier otra cláusula de la declaración **try** es dejada via **break**, **continue** or **return**

In [9]:
def dividir(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("¡división por cero!")
    else:
        print("el resultado es", result)
    finally:
        print("ejecutando la clausula finally")

In [12]:
dividir(2, 1)

el resultado es 2.0
ejecutando la clausula finally


In [11]:
dividir(2, 0)

¡división por cero!
ejecutando la clausula finally


In [10]:
divide("2", "1")

NameError: name 'divide' is not defined